# Sandbox to generate and test elasticsearch querys -> just for testing and development

start elasticsearch with .\bin\elasticsearch.bat  (you have to be in the right folder)
use the browser to check if its activated (http://127.0.0.1:9200/)

In [1]:
# Import elasticsearch packages

from elasticsearch import Elasticsearch

# Elastic search configuation

es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()

In [2]:
print('Please enter your query: ')
q = input()
print("Looking in the index for: " + q + " and showing the most relevent picture")

# make query

body_positive = {
    "from":0,
    "size":10,
    "query": {
        "bool":{
            "should":[
                {"match": { "document_text":q}}
            ],
            "filter":[
                {"range": {"sentiment": {"gt": 0}}}
            ]
        }   
    }
}
res_positive = es.search(index="final_boromir_index", body=body_positive)

body_negative = {
    "from":0,
    "size":10,
    "query": {
        "bool":{
            "should":[
                {"match": { "document_text":q}}

            ],
            "filter":[
                {"range": {"sentiment": {"lt": 0}}}
            ]            
        }   
    }
}
res_negative = es.search(index="final_boromir_index", body=body_negative)



Please enter your query: 
Looking in the index for: test and showing the most relevent picture


<ipython-input-2-8e57891e899a>:21: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res_positive = es.search(index="final_boromir_index", body=body_positive)
C:\Users\grego\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
<ipython-input-2-8e57891e899a>:38: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elastic

In [3]:
res_positive

{'took': 4209,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 5.256346,
  'hits': [{'_index': 'final_boromir_index',
    '_type': '_doc',
    '_id': 'Ia391164c0c517ec1',
    '_score': 5.256346,
    '_source': {'document_text': 'company test animal meme company test animal meme gag pampers bic dank meme gillette soundcloud beagle freedom axe company test animal meme animal maybelline bos oral sol revlon loreal revion st redken th tresemm ad company test animal know meme company test animal know meme company test animal meme animal bos acuvue aveeno axe avon wick chapstick crest diesel ove listerine giorgio pantene head shoulder dial ne redken listerine iams dank meme roundup company test animal memebase dank meme roundup company test animal memebase company test animal meme ese company test animal acuvue aveeno wick axe von almay armorall ad colgate dove crest diesel er cha

In [4]:
res_negative

{'took': 115,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 411, 'relation': 'eq'},
  'max_score': 4.5738134,
  'hits': [{'_index': 'boromir_index',
    '_type': '_doc',
    '_id': 'I0c81628165ae3517',
    '_score': 4.5738134,
    '_source': {'document_text': 'would use news june bahrain kuwait norway contact tracing apps among privacy bahrain kuwait norway invasive covid contact tracing apps world privacy security people risk amnesty international amnesty security lab contact tracing apps europe middle east north africa including apps bahrain france kuwait norway qatar human right bahrain beaware bahrain kuwait norway smittestopp apps among mass surveillance tool amnesty live live tracking user location gps coordinate central server bahrain kuwait norway run people privacy highly invasive surveillance tool go covid privacy must another government apps claudio guarnieri head amnesty international security lab nor

In [5]:
# get ID's from retrieved documents

result_positive_ids = []
results_positive = res_positive.get('hits').get('hits')

for doc in results_positive:
    id = doc.get('_id')
    result_positive_ids.append(id)

result_negative_ids = []
results_negative = res_negative.get('hits').get('hits')

for doc in results_negative:
    id = doc.get('_id')
    result_negative_ids.append(id)


# create array with all image paths from id's
import os
cwd = os.getcwd()
data_dir = cwd + "/data/images/"

# import
from PIL import Image

pictures_positive = []
for i in range(len(result_positive_ids)):
    picture_path = data_dir  + result_positive_ids[i][0:3] + "/" + result_positive_ids[i] + "/image.png"
    pictures_positive.append(picture_path)
# print(pictures)

pictures_negative = []
for i in range(len(result_negative_ids)):
    picture_path = data_dir  + result_negative_ids[i][0:3] + "/" + result_negative_ids[i] + "/image.png"
    pictures_negative.append(picture_path)


# show certain picture from pictures (default: first picture)

im = Image.open(pictures_positive[0]) 
im.show()

In [ ]:
print('positive Argumente:\n')
print(result_positive_ids)

print('\nnegative Argumente:\n')
print(result_negative_ids)

positive Argumente:

['Ie6f5910c522d0022', 'I0399627f0ac99a8d', 'I00d875ecb532bd28', 'I00e5be54c0b8ff82', 'I00e86bff8889b003', 'I00ed21daf9a80520', 'I00fc4512e71c3ca9', 'I00fcf6554f23812b', 'I0100db8cc2d6e88d', 'I01025bc48788b4b4']

negative Argumente:

['I00d8530682becc36', 'I00e7d31fd5265ece', 'I00f0530d6064b2cb', 'I0115b6db775cb940', 'I011b22837ad2dead', 'I011dc310f40b2ca9', 'I01212be044a94022', 'I0126027bc60ac694', 'I012dc6b4332ee106', 'I0347a86240f4dae2']


## Test wie sich ocr_text und image clustering einfügen lassen

In [31]:
q = 'alcohol'


#body_test_script_score = {
#    "from":0,
#    "size":10,
#    "query":{
#        "script_score":{
#          "query": {
#            "bool":{
#              "should":[
#                {"match": { "document_text":q}}
#              ],
#              "filter":[
#                {"range": {"sentiment": {"lt": 0}}}
#              ]            
#            }   
#          },
#          "script": {
#            "source": " _score * params['multiplier']['999'] + params['multiplier']['999']",  ## auf diese art kann der aktuelle scor mit dem Cluster verrechnet werden ToDo: Cluster neu zuordnen, damit das beste cluster die meiste Multiplikaiton bekommt
#            "params": {
#                "multiplier": {"0":2, "1":2, "2":2, "3":2, "4":2, "5":2, "6":2, "7":2, "8":2, "9":2, "999":0}
#            }
#          }
#        }
#      }
#}



body_test_script_score = {
    "from":0,
    "size":10,
    "query":{
        "script_score":{
          "query": {
            "bool":{
              "should":[
                {"match": { "document_text":{"query":q}}},
                {"match": {"ocr_text":{"query":q, "boost":5}}}  # ocr_text sollte einen natürlichen boost besitzen, da die texte viel kürzer sind. Vielleicht muss dieser auch abgeschwächt werden?
              ],
              "filter":[
                {"range": {"sentiment": {"lt": 0}}}
              ]            
            }   
          },
          "script": {
            "source": " _score * doc['cluster_weigth'].value"  ## auf diese art kann der aktuelle scor mit dem Cluster verrechnet werden ToDo: Cluster neu zuordnen, damit das beste cluster die meiste Multiplikaiton bekommt
          }
        }
      }
}
# doc['image_cluster'].value

res_test_script_score = es.search(index="boromir_index", body=body_test_script_score)

<ipython-input-31-ad94a1f500f3>:53: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res_test_script_score = es.search(index="boromir_index", body=body_test_script_score)


In [32]:
res_test_script_score

{'took': 474,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6580, 'relation': 'eq'},
  'max_score': 201.60614,
  'hits': [{'_index': 'boromir_index',
    '_type': '_doc',
    '_id': 'I7ed4b7e8f6cae9b2',
    '_score': 201.60614,
    '_source': {'document_text': 'content view journal content journal sign feed nature scientific report article article open access published january comparative risk assessment alcohol tobacco cannabis illicit drug using margin exposure approach dirk lachenmeier rgen rehm scientific report article number article access comparative risk assessment drug including alcohol tobacco using margin exposure moe approach wa conducted moe defined ratio toxicological threshold benchmark dose estimated human intake median lethal dose value animal experiment used benchmark dose human intake wa calculated individual scenario population based scenario moe wa calculated using probabilistic simulation be